In [1]:
import os
os.chdir('/home/cjtu/projects/essi21/essi21')
import mixing as mm
import pandas as pd
import numpy as np

f_ice = '/home/cjtu/projects/essi21/data/210702/essi00000/ice_columns_essi00000.csv'
df_ice = pd.read_csv(f_ice)
df_ice.head()

df = mm.read_crater_list()  # DataFrame, len: NC
ej_thickness_time = mm.get_ejecta_thickness_matrix(df)  # [m] shape: NY,NX,NT
ej_cols = pd.DataFrame(ej_thickness_time, columns=df.cname)

In [2]:
ej_cols.columns

Index(['Haworth', 'Shoemaker', 'Faustini', 'Amundsen', 'Cabeus', 'Cabeus B',
       'de Gerlache', 'Hedervari', 'Idel'son L', 'Unnamed 1', 'Nobile',
       'Scott', 'Scott E', 'Slater', 'Sverdrup', 'Wiechert P', 'Unnamed 2',
       'Wiechert', 'Idel'son', 'Unnamed 3', 'Wiechert U', 'Wiechert J',
       'Shackleton', 'Amundsen C'],
      dtype='object', name='cname')

In [3]:
coldtrap = 'Haworth'

def get_strat_col(ice_df, ej_df, coldtrap='Haworth'):
    strat_col = df_ice[['time']].copy()
    strat_col['ice'] = df_ice.loc[:, coldtrap].values
    strat_col['ejecta'] = ej_cols.loc[:, coldtrap].values
    strat_col.insert(1, 'depth', 0)
    strat_col.insert(2, 'label', '')

    for i, row in strat_col.iterrows():
        strat_col.loc[i+1, 'depth'] = strat_col.loc[i, 'depth'] + row.ice + row.ejecta
        if row.ejecta:
            ej_row = ej_cols.iloc[i]
            try:
                label = ej_row[ej_row == ej_row.min()].keys()[0]
            except Exception as e:
                print(e)
        elif row.ice:
            label = 'Ice'
        else:
            label = 'void'
        strat_col.loc[i, 'label'] = label
    strat_col = strat_col.drop(len(strat_col)-1)
    strat_col['depth'] = strat_col.depth.max() - strat_col.depth
    return strat_col[strat_col.label != 'void']
strat = get_strat_col(df_ice, ej_cols, coldtrap)
strat.tail()

,time,depth,label,ice,ejecta
105,3.200000e+09,5.178307,Wiechert,0.000000,0.404449
112,3.130000e+09,4.773858,Ice,0.191311,0.000000
122,3.030000e+09,4.582547,Ice,2.826662,0.000000
245,1.800000e+09,1.755885,Amundsen,0.000000,0.179705
383,4.200000e+08,1.576180,Ice,1.576180,0.000000


In [9]:
strat[['label', 'depth']][:20:-1]

,label,depth
383,Ice,1.576180
245,Amundsen,1.755885
122,Ice,4.582547
112,Ice,4.773858
105,Wiechert,5.178307
86,Ice,5.195146
85,Unnamed 3,8.342381
75,Amundsen,8.493634
71,Ice,8.672246
59,Ice,8.771650


In [12]:
strat[strat['label'] == 'Wiechert']

,time,depth,label,ice,ejecta
105,3.200000e+09,5.178307,Wiechert,0.0,0.404449


In [14]:
strat[strat['depth'] > 350]

,time,depth,label,ice,ejecta
0,4.250000e+09,984.267287,Ice,66.545370,0.000000
1,4.240000e+09,917.721917,Ice,50.207860,0.000000
2,4.230000e+09,867.514057,Ice,32.878820,0.000000
3,4.220000e+09,834.635237,Ice,47.892840,0.000000
4,4.210000e+09,786.742397,Ice,21.232365,0.000000
5,4.200000e+09,765.510032,Ice,44.287315,0.000000
6,4.190000e+09,721.222717,Ice,40.788548,0.000000
7,4.180000e+09,680.434169,Ice,43.991142,0.000000
8,4.170000e+09,636.443027,Ice,53.817410,0.000000
9,4.160000e+09,582.625617,Ice,22.966430,0.000000
